In [ ]:
import os
import copy
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import xgboost as xgb
import matplotlib.pyplot as plt
from xgboost import XGBRFRegressor
from tab_transformer_copy import TabTransformer_edit
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor

from helper_ml import train_xgb_models, load_models, generate_predictions_and_train_second_level_model
from helper_analysis import normalize_data, load_and_combine_data, format_array, load_and_normalize_data_tensor, denormalize_data

folder_path = "your_output_folder_path"  # Replace with your actual folder path
epochs_dl = 10 # For deep learning models
epochs_dl_tab = 20 # For TabNet model
epochs_ml = 200 # For machine learning models
max_files = 6 
seed = 42
scaling_type = "minmax"  # "minmax" or any other for z-score
scheduler = 0  # 1 for step, 2 for plateau, 3 for cosine
patience = 30

# Define the ratio for the train-test split
train_ratio = 0.8

# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# Define your L1 regularization strength
l1_lambda = 0.0001
l2_lambda = 0.0001

In [68]:
# Load and normalize the data
E_tensor, E_return, E_return2 = load_and_normalize_data_tensor(folder_path, "E_combo_output", max_files=max_files)
A_tensor, _, _ = load_and_normalize_data_tensor(folder_path, "A_combo_output", max_files=max_files)
J_tensor, _, _ = load_and_normalize_data_tensor(folder_path, "J_combo_output", max_files=max_files)
h_tensor, _, _ = load_and_normalize_data_tensor(folder_path, "h_combo_output", max_files=max_files)
X_tensor, _, _ = load_and_normalize_data_tensor(folder_path, "X_combo_output", max_files=max_files, scaling_type=scaling_type)
w_tensor, _, _ = load_and_normalize_data_tensor(folder_path, "w_combo_output", max_files=max_files)

# After loading the data. Make a new data tensor of size (n_samples, n_features) taking A_tensor as reference
# Starting from the 2nd index and moving by 3 beta (the angle) will be 0 and the rest will be 90 degrees (pi/2)


E_tensor.shape, A_tensor.shape, J_tensor.shape, h_tensor.shape, X_tensor.shape, w_tensor.shape

(torch.Size([81922, 15]),
 torch.Size([81922, 15]),
 torch.Size([81922, 15]),
 torch.Size([81922, 15]),
 torch.Size([81922, 45]),
 torch.Size([81922, 45]))

In [69]:
col_idx = [0,1,3,4,6,7,9,10,12,13]
beta_tensor = torch.zeros(A_tensor.shape)
type_tensor = torch.zeros(A_tensor.shape)
type_tensor[:, col_idx] = 1
beta_tensor[:, col_idx] = np.pi/2
# Reshape type_tensor to have a shape of (81922, 1)
type_tensor = type_tensor[:, :1]
# x_cont = torch.stack((A_tensor, J_tensor, h_tensor, beta_tensor, w1_tensor, X1_tensor, w2_tensor, X2_tensor, w3_tensor, X3_tensor), dim=1)
beta_tensor.shape, beta_tensor[0], type_tensor[0]

(torch.Size([81922, 15]),
 tensor([1.5708, 1.5708, 0.0000, 1.5708, 1.5708, 0.0000, 1.5708, 1.5708, 0.0000,
         1.5708, 1.5708, 0.0000, 1.5708, 1.5708, 0.0000]),
 tensor([1.]))

In [ ]:
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

# Split X_data and w_data into three separate sets of features
X1_tensor = X_tensor[:, :15].clone().detach()
X2_tensor = X_tensor[:, 15:30].clone().detach()
X3_tensor = X_tensor[:, 30:].clone().detach()
w1_tensor = w_tensor[:, :15].clone().detach()
w2_tensor = w_tensor[:, 15:30].clone().detach()
w3_tensor = w_tensor[:, 30:].clone().detach()



x_cont = torch.cat((A_tensor, J_tensor, h_tensor, beta_tensor, w1_tensor, X1_tensor, w2_tensor, X2_tensor, w3_tensor, X3_tensor), dim=1)
x_categ = torch.empty((x_cont.shape[0], 0))

x_cont_np = x_cont.numpy()
# Flatten your input tensors and concatenate them into a 2D array
X = x_cont_np
y = E_tensor

# Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Make a tensor version of the splitted data
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).clone().detach().to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).clone().detach().to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).clone().detach().to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).clone().detach().to(device)

In [ ]:
y_test.shape, y_test_tensor.shape

(torch.Size([47003, 15]), torch.Size([47003, 15]))

In [ ]:
# from sklearn.decomposition import PCA
# from sklearn.decomposition import KernelPCA


# # Assuming x_cont is your feature matrix
# x_cont_np = x_cont.numpy()  # Convert to numpy array if it's a tensor

# # Initialize PCA
# # You can specify the number of components you want to keep, e.g., PCA(n_components=10)
# pca = PCA()

# # Fit PCA to your data
# # pca.fit(x_cont_np)

# # Transform your data to the first N principal components
# # x_cont_pca = pca.transform(x_cont_np)

# # Kernel PCA
# kpca = KernelPCA(n_components=2, kernel='rbf', degree=2)  # You can adjust the number of components and the kernel
# x_cont_kpca = kpca.fit_transform(x_cont_np)

# # The explained variance ratio tells you how much information (variance) can be attributed to each principal component
# # print(pca.explained_variance_ratio_)

# # Convert the transformed data to a DataFrame
# df = pd.DataFrame(x_cont_kpca, columns=[f'PC{i+1}' for i in range(x_cont_kpca.shape[1])])

# # Create a scatter plot matrix
# sns.pairplot(df, vars=['PC1', 'PC2', 'PC3', 'PC4'])  # You can adjust the components you want to plot
# plt.show()

In [ ]:
# Initialize your XGBoost regressors
models = [xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.5, learning_rate = 0.1,
                max_depth = 15, alpha = 5, n_estimators = 200) for _ in range(y.shape[1])]

# Fit the models to the training data and make predictions
y_pred = np.zeros_like(y_test)
for i, model in enumerate(models):
    model.fit(X_train, y_train[:, i])
    y_pred[:, i] = model.predict(X_test)

    # Save the trained model
    model.save_model(f'xgb_model_{i}.json')

# Compute the RMSE of the predictions
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))

# Initialize an empty list to hold the models
models = []

# Load the models
for i in range(y.shape[1]):
    model = xgb.XGBRegressor()
    model.load_model(f'xgb_model_{i}.json')
    models.append(model)


# Convert the predictions to a tensor
y_pred_tensor = torch.tensor(y_pred)

# Denormalize the predictions
y_pred_denorm = denormalize_data(y_pred_tensor, E_return, E_return2, scaling_type)
y_pred_denorm_np = y_pred_denorm.numpy()

# Denormalize the actual values
y_test_tensor = torch.tensor(y_test)
y_test_denorm = denormalize_data(y_test_tensor, E_return, E_return2, scaling_type)
y_test_denorm_np = y_test_denorm.numpy()

# Calculate MAPE
mape = np.mean(np.abs((y_test_denorm_np - y_pred_denorm_np) / np.abs(y_test_denorm_np))) * 100

# Calculate MdAPE
mdape = np.median(np.abs((y_test_denorm_np - y_pred_denorm_np) / np.abs(y_test_denorm_np))) * 100

# Calculate the difference
difference = y_test_denorm_np - y_pred_denorm_np

In [ ]:
### Training the models

# Train the models
y_pred = train_xgb_models(X_train, y_train, X_test, y_test)

# Load the models
models = load_models(y.shape[1], use_gpu=True)

# Predict the test set (if you have not trained the models in this session)
y_pred = np.zeros_like(y_test)
for i, model in enumerate(models):
    y_pred[:, i] = model.predict(X_test)

# Convert the predictions to a tensor
y_pred_tensor = torch.tensor(y_pred)

# Denormalize the predictions
y_pred_denorm = denormalize_data(y_pred_tensor, E_return, E_return2, scaling_type)
y_pred_denorm_np = y_pred_denorm.numpy()

# Denormalize the actual values
y_test_tensor = torch.tensor(y_test)
y_test_denorm = denormalize_data(y_test_tensor, E_return, E_return2, scaling_type)
y_test_denorm_np = y_test_denorm.numpy()

# Calculate MAPE
mape = np.mean(np.abs((y_test_denorm_np - y_pred_denorm_np) / np.abs(y_test_denorm_np))) * 100

# Calculate MdAPE
mdape = np.median(np.abs((y_test_denorm_np - y_pred_denorm_np) / np.abs(y_test_denorm_np))) * 100

# Calculate the difference
difference = y_test_denorm_np - y_pred_denorm_np

In [ ]:
# Print the results
print("MAPE:", mape)
print("MdAPE:", mdape)
print("Difference:", difference[123])
print("Original:", y_test_denorm[123].numpy(), len(y_pred))
print("Denormalized predictions:", y_pred_denorm[123].numpy(), len(y_pred_denorm))

MAPE: 7.362738996744156
MdAPE: 2.2229790687561035
Difference: [  5604208.  -4466640.         0.  -2811184.   -506384.         0.
   -112224.  -3357280.         0. -20054432.   3808960.         0.
  14474864.   4909956.         0.]
Original: [2.100e+08 1.050e+08 2.100e+08 1.575e+08 1.050e+08 2.100e+08 2.100e+08
 2.100e+08 2.100e+08 1.575e+08 2.100e+08 2.100e+08 2.100e+08 5.250e+07
 2.100e+08] 16385
Denormalized predictions: [2.0439579e+08 1.0946664e+08 2.1000000e+08 1.6031118e+08 1.0550638e+08
 2.1000000e+08 2.1011222e+08 2.1335728e+08 2.1000000e+08 1.7755443e+08
 2.0619104e+08 2.1000000e+08 1.9552514e+08 4.7590044e+07 2.1000000e+08] 16385


In [ ]:
# Load model itself
model = TabTransformer_edit(
    categories=(),
    num_continuous=150,               # continuous features
    dim=150,                          # dimension, paper set at 32
    dim_out=15,                      # 15 due to the shape of E_data
    depth=3,                         # depth, paper recommended 6 (9 worked better by .02%)
    heads=15,                         # heads, paper recommends 8
    attn_dropout=0.6,                # post-attention dropout
    ff_dropout=0.6,                  # feed forward dropout
    mlp_hidden_mults=(4,2),         # relative multiples of each hidden dimension of the last mlp to logits
    mlp_act=nn.ReLU6(),               # activation for final mlp, defaults to relu, but could be anything else (selu, ELU, PReLU, Tanh, Sigmoid)
    mlp_dropout=0.05,
)

model.load_state_dict(torch.load('model_weights_withfreq4.pth'))
model = model.to(device)

# Test the model
model.eval()
with torch.no_grad():
    pred_test_dl = model(x_categ[:X_test.shape[0]], X_test_tensor)

# Move the predictions back to the CPU for further processing
pred_test_dl = pred_test_dl.to("cpu")
y_test = y_test.to("cpu")

# Denormalize the E_tensor
E_test_denorm = denormalize_data(y_test, E_return, E_return2, scaling_type)
E_test_denorm_np = E_test_denorm.numpy()
# Denormalize the predictions
pred_test_denorm_dl = denormalize_data(pred_test_dl, E_return, E_return2, scaling_type)
pred_test_denorm_np_dl = pred_test_denorm_dl.numpy()

# MAPE
mape_dl = np.mean(np.abs((E_test_denorm_np - pred_test_denorm_np_dl) / np.abs(E_test_denorm_np))) * 100
# MdAPE
mdape_dl = np.median(np.abs((E_test_denorm_np - pred_test_denorm_np_dl) / np.abs(E_test_denorm_np))) * 100

# Calculate the difference
difference_dl = E_test_denorm_np - pred_test_denorm_np_dl

In [ ]:
# Create a copy of the model
model2 = copy.deepcopy(model)

# Load different weights into model2
model2.load_state_dict(torch.load('model_weights_withfreq4.pth'))

# Move model2 to the device
model2 = model2.to(device)

# Test model2
model2.eval()
with torch.no_grad():
    pred_test_dl2 = model2(x_categ[:X_test.shape[0]], X_test_tensor)

# Move the predictions back to the CPU for further processing
pred_test_dl2 = pred_test_dl2.to("cpu")

# Denormalize the predictions
pred_test_denorm_dl2 = denormalize_data(pred_test_dl2, E_return, E_return2, scaling_type)
pred_test_denorm_np_dl2 = pred_test_denorm_dl2.numpy()

# MAPE
mape_dl2 = np.mean(np.abs((E_test_denorm_np - pred_test_denorm_np_dl2) / np.abs(E_test_denorm_np))) * 100
# MdAPE
mdape_dl2 = np.median(np.abs((E_test_denorm_np - pred_test_denorm_np_dl2) / np.abs(E_test_denorm_np))) * 100

# Calculate the difference
difference_dl2 = E_test_denorm_np - pred_test_denorm_np_dl2

In [76]:
# Print the results
sample = 123
print("MAPE:", mape_dl, mape_dl2)
print("MdAPE:", mdape_dl, mdape_dl2)
print("Difference:", difference_dl2[sample])
print("Original:", E_test_denorm[sample].numpy())

MAPE: 5.106492340564728 4.091095924377441
MdAPE: 2.2805027663707733 1.7494704574346542
Difference: [ 6.8607200e+06 -5.1106880e+06  7.0400000e+02 -1.0403296e+07
 -7.0272400e+06  1.1840000e+03  6.4405920e+06  2.7732000e+06
  9.4400000e+02  5.1756800e+06  8.6895360e+06  1.7600000e+02
  2.2118432e+07 -2.9237120e+06  6.8800000e+02]
Original: [2.100e+08 1.050e+08 2.100e+08 1.575e+08 1.050e+08 2.100e+08 2.100e+08
 2.100e+08 2.100e+08 1.575e+08 2.100e+08 2.100e+08 2.100e+08 5.250e+07
 2.100e+08]


In [ ]:
### ENSEMBLE MODEL 1 - Naive Method ###
# Calculate the element-wise MAPE
mape_lgbm = np.abs((y_test_denorm_np - y_pred_denorm_np) / np.abs(y_test_denorm_np)) + 1e-10
mape_dl = np.abs((y_test_denorm_np - pred_test_denorm_np_dl) / np.abs(y_test_denorm_np)) + 1e-10
mape_dl_2 = np.abs((y_test_denorm_np - pred_test_denorm_np_dl2) / np.abs(y_test_denorm_np)) + 1e-10

# Calculate the variance of the MAPE element-wise
var_lgbm = np.var(mape_lgbm)
var_dl = np.var(mape_dl)
var_dl_2 = np.var(mape_dl_2)

# Calculate the weights based on the inverse of the variance
weight_lgbm = 1 / var_lgbm
weight_dl = 1 / var_dl
weight_dl_2 = 1 / var_dl_2

# Normalize the weights so they sum to 1 for each element
total_weight = weight_lgbm + weight_dl + weight_dl_2
weight_lgbm /= total_weight
weight_dl /= total_weight
weight_dl_2 /= total_weight

# Perform the ensemble
ensemble_prediction = weight_lgbm * y_pred_denorm_np + weight_dl * pred_test_denorm_np_dl + weight_dl_2 * pred_test_denorm_np_dl2
mape_ensemble = np.mean(np.abs((y_test_denorm_np - ensemble_prediction) / np.abs(y_test_denorm_np))) * 100
mdape_ensemble = np.median(np.abs((y_test_denorm_np - ensemble_prediction) / np.abs(y_test_denorm_np))) * 100

# Calculate the difference
difference_ensemble = y_test_denorm_np - ensemble_prediction

In [78]:
sample = 123
print("MAPE:", mape_ensemble)
print("MdAPE:", mdape_ensemble)
print("Difference:", difference_ensemble[sample])
print("Prediction:", y_pred_denorm[sample].numpy())
print("Original:", y_test_denorm[sample].numpy(), len(y_pred))

MAPE: 3.849160298705101
MdAPE: 1.6662705689668655
Difference: [ 1.7260960e+06  5.0040800e+05 -9.4400000e+02 -1.1279296e+07
  2.5038400e+05 -1.0240000e+03  5.0848160e+06  4.7190400e+06
  5.2800000e+02 -3.2168000e+06  8.1463680e+06 -9.8080000e+03
  2.1986560e+07 -6.6765600e+05 -1.0304000e+04]
Prediction: [2.0439579e+08 1.0946664e+08 2.1000000e+08 1.6031118e+08 1.0550638e+08
 2.1000000e+08 2.1011222e+08 2.1335728e+08 2.1000000e+08 1.7755443e+08
 2.0619104e+08 2.1000000e+08 1.9552514e+08 4.7590044e+07 2.1000000e+08]
Original: [2.100e+08 1.050e+08 2.100e+08 1.575e+08 1.050e+08 2.100e+08 2.100e+08
 2.100e+08 2.100e+08 1.575e+08 2.100e+08 2.100e+08 2.100e+08 5.250e+07
 2.100e+08] 16385


In [ ]:
# Ensemble model
# Generate predictions from the LightGBM models on the training data
y_pred_train_lgbm = np.column_stack([model.predict(X_train) for model in models])

# Generate predictions from the Deep Learning model on the training data
model.eval()
with torch.no_grad():
    y_pred_train_dl = model(x_categ[:X_test.shape[0]],X_train_tensor).cpu().numpy()

# Stack the predictions together to form a new feature set for the second-level model
X_train_level2 = np.column_stack([y_pred_train_lgbm, y_pred_train_dl])

# Convert the labels to a numpy array
y_train_np = y_train_tensor.cpu().numpy()

# Train the second-level model
dtrain = xgb.DMatrix(X_train_level2, label=y_train_np)
params = {'objective': 'reg:squarederror', 'eval_metric': 'mae', 'eta': 0.1, 'max_depth': 6}
model_level2 = xgb.train(params, dtrain, num_boost_round=1000)

# Generate predictions from the LightGBM models on the test data
y_pred_test_lgbm = np.column_stack([model.predict(X_test) for model in models])

# Generate predictions from the Deep Learning model on the test data
model.eval()
with torch.no_grad():
    y_pred_test_dl = model(x_categ[:X_test.shape[0]],X_test_tensor).cpu().numpy()

# Stack the predictions together to form a new feature set for the second-level model
X_test_level2 = np.column_stack([y_pred_test_lgbm, y_pred_test_dl])

# Generate the final prediction from the second-level model
dtest = xgb.DMatrix(X_test_level2)
y_pred_final = model_level2.predict(dtest)

# Denormalize the final prediction
y_pred_final_denorm = denormalize_data(torch.tensor(y_pred_final), E_return, E_return2, scaling_type).numpy()

# MAPE
mape_ensemble = np.mean(np.abs((y_test_denorm_np - y_pred_final_denorm) / np.abs(y_test_denorm_np))) * 100
# MdAPE
mdape_ensemble = np.median(np.abs((y_test_denorm_np - y_pred_final_denorm) / np.abs(y_test_denorm_np))) * 100

# Calculate the difference
difference_ensemble = y_test_denorm_np - y_pred_final_denorm

In [ ]:
# Call the function for the first time and save a pickle
y_pred_final = generate_predictions_and_train_second_level_model(X_train, y_train, X_test, y_test, num_models=len(models), model=model, x_categ=x_categ, 
                                                                 X_train_tensor=X_train_tensor, X_test_tensor=X_test_tensor, E_return=E_return, 
                                                                 E_return2=E_return2, scaling_type=scaling_type, name="xgb_model", train_second_level_model=True,
                                                                 models=models)

# Denormalize the final prediction
y_pred_final_denorm = denormalize_data(torch.tensor(y_pred_final), E_return, E_return2, scaling_type).numpy()

# MAPE
mape_ensemble = np.mean(np.abs((E_test_denorm_np - y_pred_final_denorm) / np.abs(E_test_denorm_np))) * 100
# MdAPE
mdape_ensemble = np.median(np.abs((E_test_denorm_np - y_pred_final_denorm) / np.abs(E_test_denorm_np))) * 100

# Calculate the difference
difference_ensemble = E_test_denorm_np - y_pred_final_denorm

In [80]:
# Call the function
y_pred_final = generate_predictions_and_train_second_level_model(X_train, y_train, X_test, y_test, num_models=len(models), model=model, x_categ=x_categ, 
                                                                 X_train_tensor=X_train_tensor, X_test_tensor=X_test_tensor, E_return=E_return, 
                                                                 E_return2=E_return2, scaling_type=scaling_type, name="xgb_model", train_second_level_model=False, 
                                                                 models=models, second_name="xgb_model_level2_8files")

# Denormalize the final prediction
y_pred_final_denorm = denormalize_data(torch.tensor(y_pred_final), E_return, E_return2, scaling_type).numpy()

# MAPE
mape_ensemble = np.mean(np.abs((E_test_denorm_np - y_pred_final_denorm) / np.abs(E_test_denorm_np))) * 100
# MdAPE
mdape_ensemble = np.median(np.abs((E_test_denorm_np - y_pred_final_denorm) / np.abs(E_test_denorm_np))) * 100

# Calculate the difference
difference_ensemble = E_test_denorm_np - y_pred_final_denorm

print("MAPE:", mape_ensemble)
print("MdAPE:", mdape_ensemble)
print("Difference:", difference_ensemble[123])
print("Prediction from ensemble:", y_pred_final_denorm[123], len(y_pred_final_denorm))
print("Original:", y_test_denorm[123].numpy(), len(y_pred))

MAPE: 1.5224144794046879
MdAPE: 0.35021714866161346
Difference: [ 2.2896000e+04  9.8304000e+04  4.8000000e+01 -5.7658240e+06
  1.4878688e+07  4.8000000e+01  5.6284800e+05 -2.4715840e+06
  4.8000000e+01  5.2822400e+05 -2.4139200e+05  4.8000000e+01
 -2.3846400e+06 -8.9351200e+05  4.8000000e+01]
Prediction from ensemble: [2.0997710e+08 1.0490170e+08 2.0999995e+08 1.6326582e+08 9.0121312e+07
 2.0999995e+08 2.0943715e+08 2.1247158e+08 2.0999995e+08 1.5697178e+08
 2.1024139e+08 2.0999995e+08 2.1238464e+08 5.3393512e+07 2.0999995e+08] 16385
Original: [2.100e+08 1.050e+08 2.100e+08 1.575e+08 1.050e+08 2.100e+08 2.100e+08
 2.100e+08 2.100e+08 1.575e+08 2.100e+08 2.100e+08 2.100e+08 5.250e+07
 2.100e+08] 16385


In [ ]:
# Using a different DL model
print("MAPE:", mape_ensemble)
print("MdAPE:", mdape_ensemble)
print("Difference:", difference_ensemble[123])
print("Prediction from ensemble:", y_pred_final_denorm[123], len(y_pred_final_denorm))
print("Original:", y_test_denorm[123].numpy(), len(y_pred))

MAPE: 1.1730022728443146
MdAPE: 0.0678399985190481
Difference: [-2.5177280e+06  4.4444000e+06 -4.1616000e+04 -1.1686656e+07
  2.1532800e+05 -6.5600000e+04  5.8907200e+06  1.1599616e+07
  6.4960000e+03 -1.1539632e+07  5.7415040e+06  4.6320000e+04
  1.6516192e+07  5.4013000e+06  1.4304000e+04]
Prediction from ensemble: [2.1003094e+08 1.0501678e+08 2.0999995e+08 1.5844226e+08 1.0523671e+08
 2.0999995e+08 2.0976867e+08 2.1017341e+08 2.0999995e+08 1.5790074e+08
 2.1009451e+08 2.0999995e+08 2.0946392e+08 5.2994408e+07 2.0999995e+08] 16385
Original: [2.100e+08 1.050e+08 2.100e+08 1.575e+08 1.050e+08 2.100e+08 2.100e+08
 2.100e+08 2.100e+08 1.575e+08 2.100e+08 2.100e+08 2.100e+08 5.250e+07
 2.100e+08] 16385
